This files is for training/generating HMMs with **two states**.

In [7]:
from hmmlearn import hmm
import pandas as pd
import os
import numpy as np
import pickle

In [8]:
"""
Appends the error values of the file(sequence) to X and the length of the sequence to the array lengths. 

@param filename: name of file/sequence
@param X: array of error values for all practice opportunities of one practice mode
@param lengths: array of lengths of the individual sequences

"""
def get_sequence(filename, X, lengths):
    file = pd.read_csv(filename)
    for i, row in file.iterrows():
        X.append(list(row[4:]))
    lengths.append(len(file))
    return X, lengths

In [9]:
"""
Tries to find the best HMM, by returning the model with the highest score/log probability. Note that this is
not necessarily the most reasonable, most-fitting model to the data, so exploring other models is important!

@param filename: practice mode

@return modelBest: model with the highest logProb
@return logProbMax: the highest logProb
@return PBest: the transition matrix corresponding to the best model
"""
def findBestHMM(filename):
    logProbMax = 0
    modelBest = None
    PBest = None
    for i in range(10):
        try:
            model, mus, sigmas, P, logProb, convergence = fitHMM(X, lengths)
            if logProb > logProbMax and convergence.converged == True:
                logProbMax = logProb
                modelBest = model
                PBest = P 
        except:
            continue
    with open(filename + ".pkl", "wb") as file: pickle.dump(modelBest, file)
    return modelBest, logProbMax, PBest

In [10]:
"""
Generates a Hidden Markov Model. The startprobability and initial transition probability should 
be varied between retries.

@return model:
@return mus: means of states
@return sigmas: covariance matrix
@return P: transition probability
@return logProb:score of model
@return convergence: monitors convergence of model
"""
def fitHMM(X, lengths):
    # fit Gaussian HMM to Q
    model = hmm.GaussianHMM(n_components=2, n_iter=1000, init_params="cm")
    
    model.startprob_ = np.array([1.0, 0.0]) # same initial state
    model.transmat_ = np.array([[0.5, 0.5], 
                                [0.0, 1.0]])
     
    model.fit(X, lengths)
    # classify each observation as state 0 or 1
    #hidden_states = model.predict(X)
 
    # find parameters of Gaussian HMM
    convergence = model.monitor_
    mus = np.array(model.means_)
    sigmas = np.array(np.sqrt(np.array([np.diag(model.covars_[0]),np.diag(model.covars_[1])])))
    P = np.array(model.transmat_)
 
    # find log-likelihood of Gaussian HMM
    logProb = model.score(X, lengths)
 
    return model, mus, sigmas, P, logProb, convergence #hidden_states

In [11]:
"""
Main: Generates models for one practice mode (directory) or iteratively for all (don't forget to save them
in that case).
"""
#for directory in ["None", "right hand", "single note", "slower", "split hands", "both hands"]:
directory = "both hands"
lengths = []
X = []
for filename in os.listdir(directory):
    X, lengths = get_sequence(directory + "/" + filename, X, lengths)
model, mus, sigmas, P, logProb, convergence = fitHMM(X, lengths) #directory equals practice mode
#model, logProb, P = findBestHMM(directory)
print("/n", directory)
print("P", P)
print("logProb", logProb)
#print("means", mus)
print("convergences", convergence)
print("")


/n both hands
P [[0.77777808 0.22222192]
 [0.         1.        ]]
logProb 867.5739709456111
convergences ConvergenceMonitor(
    history=[347.57388404670047, 845.2975483347807, 867.5738946786869, 867.5739709057594],
    iter=4,
    n_iter=1000,
    tol=0.01,
    verbose=False,
)



In [12]:
"""For saving models:"""
#with open("both hands2.pkl", "wb") as file: pickle.dump(model, file)

'For saving models:'

## Normalise with error values

In [17]:
"""
Calculates and prints the state with the min/max error value(observation/performance errors) according
to the raw errors and another error metric, which adds 1 to each state if its the min/max error value in 
that error section(eg pitch).
This is then used to normalise the model. That means, that the transition matrix is saved in a way, that
column/row 0 is always the unmastered state (high error) and column/row 1 is always the mastered state 
(low error).
"""
filename = "single note1.pkl"
with open("BestTwoStates/"  + filename, "rb") as file:
            model = pickle.load(file)
print("Transition matrix")
print(model.transmat_)

#print("Model means")
#print(model.means_)
mus = model.means_

state0 = mus[0][5] + mus[0][11]
state1 = mus[1][5] + mus[1][11]

print([state0, state1])

max_error = np.argmax([state0, state1])
min_error = np.argmin([state0, state1])

new_max, new_min = new_error_comp(mus)

print("state with min_error:", min_error)
print("state with max error:", max_error)

print("state with new min_error:", new_min)
print("state with new max error:", new_max)
#means
#covariance
#transition
indizes = []


if new_min == 0:
    indizes = [1,0]
    model.transmat_ = swap_row(swap_column(model.transmat_, indizes), indizes)
    model.means_ = swap_row(model.means_, indizes)


print("Transition matrix after")
print(model.transmat_)

        
#with open("left hand4_new.pkl", "wb") as file: pickle.dump(model, file)


Transition matrix
[[0.95067437 0.04932563]
 [0.         1.        ]]
[3.232643636308577, 2.840237179012517]
min 0 0
state with min_error: 1
state with max error: 0
state with new min_error: 0
state with new max error: 0
Transition matrix after
[[1.         0.        ]
 [0.04932563 0.95067437]]


In [14]:
"""
Calculates another min/max state in regards to errors, as described above.
"""
def new_error_comp(mus):
    state0 = state1 = 0
    for i in range(len(mus[0])):
        emax = np.argmax([mus[0][i], mus[1][i]])
        emin = np.argmin([mus[0][i], mus[1][i]])
        if emax == 0:
            state0 += 1
        elif emax == 1:
            state1 += 1
        if emin == 0:
            state0 -= 1
        elif emin == 1:
            state1 -= 1
    new_max = np.argmax([state0, state1])
    new_min = np.argmin([state0, state1])
    print("min", state0, state1)
    return new_max, new_min

In [15]:
def swap_column(arr, indizes):
    arr[:, [0,1]] = arr[:, indizes]
    return arr

In [16]:
def swap_row(arr, indizes):
    arr[[0,1]] = arr[indizes]
    return arr